In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import random
# Goal A - [0,11],Goal B -[2,9],Goal C -[6,7]
goal_c=7
goal_r=6
# core algorithm
epsilon_value = 0.1   # assume
alpha = 0.5  #learning rate
gamma = 0.9  #discount rate
wind_prob = 0.5  
episodes = 5000 
runs = 50  
M = 10000
lmbda=0.9
steps=np.zeros((runs,episodes))
G_all=np.zeros((runs,episodes))
dum_a = np.array([0,1,2,3])
grid_rewards = np.array([[0, 0, 0, 0,  0,  0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0,  0,  0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, -1,  -1,  -1, -1, -1, -1, 0, 0, 0],
              [0, 0, 0, -1,  -2,  -2, -2, -2, -1, 0, 0, 0],
              [0, 0, 0, -1,  -2,  -3, -3, -2, -1, 0, 0, 0],
              [0, 0, 0, -1,  -2,  -3, -2, -2, -1, 0, 0, 0],
              [0, 0, 0, -1,  -2,  -3, -2, -1, -1, 0, 0, 0],
              [0, 0, 0, -1,  -2,  -2, -2, -1, 0, 0, 0, 0],
              [0, 0, 0, -1,  -1,  -1, -1, -1, 0, 0, 0, 0],
              [0, 0, 0, 0,  0,  0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0,  0,  0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0,  0,  0, 0, 0, 0, 0, 0, 0]]).astype("float32")
grid_rewards[goal_r,goal_c]=10

In [2]:
def statechangefn(act_sel,s_r,s_c):
    dum_b=set(dum_a) - set([act_sel])
    pr_action = np.zeros_like(dum_a).astype(float)
    pr_action[act_sel,]=gamma
    pr_action[list(dum_b)]=(1-gamma)/3
    cum_prob = np.cumsum(pr_action)
    sel_direction= np.digitize(random.uniform(0,1),cum_prob)
    if sel_direction == 0:
        s_new_r = s_r -1 
        s_new_c = s_c
    elif sel_direction == 1:
        s_new_r = s_r  
        s_new_c = s_c + 1         
    elif sel_direction == 2:
        s_new_r = s_r + 1
        s_new_c = s_c        
    elif sel_direction == 3:
        s_new_r = s_r  
        s_new_c = s_c - 1
        
    # States after hitting wall remains unchanged (ROWS)
    if s_new_r == -1:
        s_new_r = 0
    elif s_new_r == 12:
        s_new_r = 11
        
    #States after hitting wall remains unchanged (COLUMN)
    if s_new_c == -1:
        s_new_c = 0
    elif s_new_c == 12:
        s_new_c = 11
        
    # wind effect
    if random.uniform(0,1)< wind_prob:
        s_new_c = s_new_c + 1
        
    # after hitting wall remains unchanged (COLUMN)
    if s_new_c == 12:
        s_new_c = 11
        
    return s_new_r, s_new_c

In [ ]:
for r in range(runs):
    print(r)
    #initial action value function
    q_value = np.random.randint(0,10, (4, 12, 12)).astype(float)/10
    q_value[:,goal_r,goal_c]=0
    for ep in range(episodes):
        # select one among the four start states
        start_r=np.random.choice([5, 6, 10, 11])
        start_c = 0 # first column
        s_r = start_r
        s_c = start_c
        G = 0 
        e=np.zeros([4,12,12])
        if np.random.uniform(0,1)> epsilon_value:
            act_sel=np.argmax(q_value[:,s_r, s_c])
        else:
            act_sel=np.random.choice([0,1, 2, 3])
                
            for k in range(M):
                    
                if s_r == goal_r and s_c == goal_c: 
                    break
                else:
                    [s_new_r,s_new_c] = statechangefn(act_sel,s_r, s_c)
                    if np.random.uniform(0,1)> epsilon_value:
                        act_next=np.argmax(q_value[:,s_new_r,s_new_c])
                    else:
                        act_next=np.random.choice([0,1, 2, 3])
                        
                    delta= grid_rewards[s_new_r, s_new_c]  + gamma*(q_value[act_next,s_new_r, s_new_c]) - q_value[act_sel,s_r, s_c]
                    e[act_sel,s_r, s_c]=e[act_sel,s_r, s_c]+1
                    for i in range(0,12):
                        for j in range(0,12):
                            for l in range(0,4):
                                q_value[l,i,j] = q_value[l,i,j] + alpha*delta*e[l,i,j]
                                e[l,i,j]=gamma*lmbda*e[l,i,j]
                    s_r = s_new_r
                    s_c = s_new_c
                    act_sel=act_next
                    G = G + gamma**(k)*grid_rewards[s_new_r,s_new_c]
            steps[r,ep]=k
            G_all [r,ep] = G
avg_steps = np.mean(steps,0)
avg_G = np.mean(G_all,0)  

0
1
2
3
4
5
6


In [ ]:
plt.plot(avg_G,color="green")
plt.title('SARSA (lambda=0.9)Goal:C', fontsize=16)
plt.xlabel('No of Episodes')
plt.ylabel('Cummulative Discounted Reward')
plt.savefig('C:/Users/Venkatachalam S/Desktop/POLICY/SLGA5.jpg',bbox_inches='tight',dpi=300)
plt.show()

In [ ]:
plt.plot(avg_steps)
plt.title('SARSA (lambda=0.9)Goal:C', fontsize=16)
plt.xlabel('No of Episodes')
plt.ylabel('Cummulative Discounted Reward')
plt.savefig('C:/Users/Venkatachalam S/Desktop/POLICY/SLGA6.jpg',bbox_inches='tight',dpi=300)
plt.show()